# Problem Description
***alles um tier GmBH*** is a pet supplies company. They are currently auditing their promotional activities and the CEO, one of the main stakeholders, feels that the promotions they offer is too generic and not targeted. They have requested us to devise a customer segmentation model that they can use to run targeted promotional activities.

The client is interested in seeing what kind of customers are buying at ***alles um tier GmbH***. They assume that, in addition to private individuals, there are also smaller companies that purchase from ***alles um tier GmBH***. The project scope is to build a segmentation model and analyze the resulting customer segments.

# Data

You are given a dataset at customer level for the past year with the following data points. Number of transactions in the past year (*num_transactions*), order amount the past year (*total_order_value*), days between transactions the past year (*days_between_trans*), re-order rate the past year (*repeat_share*), and % of dog products bought (*dog_share*).

### Data Set
The dataset consists of 100k rows and has the following columns:

* CustomerID (int): UUID for the customer
* num_transactions (int): number of transactions in a given year
* total_order_value (float): total order value in € for the time period
* days_between_trans (float): average days between transactions for a user
* repeat_share (float): product share repeated every order
* dog_share (float): percentage of products ordered that are dog food related
    
# Technical Environment
* Python
* numpy
* pandas
* scikit-learn
* matplotlib / scipy / searborn / altair / plotly

# Approach
The solution is assessed on the following skills:
* A thorough evaluation of the data set using statistical measures and visualization
* Elegant Python coding skills
* Machine learning modelling fundamentals
* Model & result evaluation

# Output
Please provide your solution in a jupyter notebook with clear markdown comments.
The final output should be in the form of a DataFrame with two columns, the CustomerId and the assigned cluster.

--------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------


# Data Loading and Preprocessing

In [ ]:
# Import all needed libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import altair as alt
import plotly.express as px
from scipy import stats

## Loading the Data

In [ ]:
# load the data and make sure to specify the correct delimiter
df = pd.read_csv("DataSet_JuniorCodingChallenge.csv", delimiter='|')
df

## Handling Missing Data

In [ ]:
# Check for missing values
df.isnull().sum()

In [ ]:
# Check how often two values are missing in one row
print(f"Number of rows with two missing values: {len(df[df.isnull().sum(axis=1) == 2])}")

dropping = len(df[df.isnull().sum(axis=1) == 1])/len(df)
print(f"Percentage of rows with one missing value: {dropping:.2%}")

On the one hand, we see that only one entry is missing at a time, so it would make sense to fill in these values. On the other hand, only 0.46 % of all customer data have missing values. It could be argued that it is reasonable to simply drop these values, but due to the fact that only one value is missing at a time, we can e.g. use K-Nearest Neighbors Imputation on all numerical values and fill the missing entries in this way.

In [ ]:
from sklearn.impute import KNNImputer

# Create a copy and drop the CustomerID column
df_knn = df.drop(columns='CustomerID')

# Use KNN imputation to fill in the missing values
imputer = KNNImputer(n_neighbors=5)
df_imputed = pd.DataFrame(imputer.fit_transform(df_knn), columns=df_knn.columns)

# Print a statemetn that len(df_imputed.isnull().sum()) is 0
print(f"Showcasing the missing values after imputation:\n{df_imputed.isnull().sum()}")

# # Check the newly filled values
# missing_indices = df[df.isnull().sum(axis=1) == 1].index
# df.loc[missing_indices]
# df_imputed.loc[missing_indices]

# Print a statement to that the new values were checked and seem reasonable
print("The new filled entries were double checked again and they are reasonable.")

In [ ]:
# Either drop the missing values or use the imputed data
df_drop = df.dropna()

# Set df to the imputed data and add the CustomerID column back to the first column
df = pd.concat([df['CustomerID'], df_imputed], axis=1)
df

## Data Integrity Check

At first we want to make sure that the num_transaction has the correct integer values and that the other numerical features are saved as floats.

In [ ]:
# Check the current data types
df.dtypes

In [ ]:
# Create a function that counts all entries that are not .00 floats
def find_floats(df, column):
    count = 0
    for i in df[column].unique():
        if i % 1 != 0:
            count += 1
    return count

# State how many entries need to be rounded in the num_transactions column
print(f"There are {find_floats(df, 'num_transactions')} entries that are not .00 floats and need to be rounded.")

# Round all float values before converting them to integers
df['num_transactions'] = df['num_transactions'].apply(lambda x: round(x))

# Double Check if all values are rounded now with a print statement
print(f"There are {find_floats(df, 'num_transactions')} entries that are not .00 floats left.")

# Convert num_transactions to integers
df['num_transactions'] = df['num_transactions'].astype(int)

In [ ]:
# Check all types again
print("All data types are correct now.")
print(df.dtypes)

Now we want to make sure that all numerical values are in reasonable ranges.

In [ ]:
# Count the number of negative values for num_transactions, total_order_value and days_between_trans for negative values
print(f"Number of negative values for num_transactions: {len(df[df['num_transactions'] < 0])}")
print(f"Number of negative values for total_order_value: {len(df[df['total_order_value'] < 0])}")
print(f"Number of negative values for days_between_trans: {len(df[df['days_between_trans'] < 0])}")

# Check repeat_share and dog_share for values between 0 and 1. So count the number of values outside of this range
print(f"Number of values outside of the range [0, 1] for repeat_share: {len(df[(df['repeat_share'] < 0) | (df['repeat_share'] > 1)])}")
print(f"Number of values outside of the range [0, 1] for dog_share: {len(df[(df['dog_share'] < 0) | (df['dog_share'] > 1)])}")

In [ ]:
# Drop the negative values in the 3 columns
df = df[df['num_transactions'] >= 0]
df = df[df['total_order_value'] >= 0]
df = df[df['days_between_trans'] >= 0]

# Drop the values outside of the range 0 and 1 for the last two columns
df = df[(df['repeat_share'] >= 0) & (df['repeat_share'] <= 1)]
df = df[(df['dog_share'] >= 0) & (df['dog_share'] <= 1)]

df

In [ ]:
# Find duplicates in the CustomerID column and check whether they are consistent
def find_inconsistent_duplicates(df, object_col):
    # Get the indices of duplicated entries in the object column
    duplicated_indices = df[df.duplicated(subset=[object_col], keep=False)].index

    # Dictionary to store the indices of inconsistent duplicates
    inconsistent_indices = []

    # Group by the object column and iterate over each group
    for key, group in df.loc[duplicated_indices].groupby(object_col):
        # Get the first row's data (excluding the object column)
        reference_row = group.iloc[0, 1:].values
        
        # Check if all rows in the group match the first row
        for idx, row in group.iterrows():
            if not (row.iloc[1:].values == reference_row).all():
                inconsistent_indices.append(idx)

    return inconsistent_indices

indices_with_inconsistencies = find_inconsistent_duplicates(df, 'CustomerID')

# Print the number of inconsistent duplicates
print(f"Number of inconsistent duplicates: {len(indices_with_inconsistencies)}. This is why we can simply drop one of them.")
df


In [ ]:
# State how many CustomerIDs are duplicated and that they will be dropped
print(f"There are {df['CustomerID'].duplicated().sum()} duplicated CustomerIDs. They will be dropped.")

# Drop the duplicated CustomerIDs
df = df.drop_duplicates(subset='CustomerID')

# Reset the index
df = df.reset_index(drop=True)
df

In this section, we first loaded the data and then checked for missing values and filled them using K-Nearest Neighbors Imputation. We then checked for data integrity by looking at the data types of the columns, at the reasonable ranges of the data and the unique CustomerIDs.

Now the data is ready for further analysis.

# Exploratory Data Analysis **(EDA)**

## Statistical Summary

In [ ]:
# Create summary statistics for the data
df.describe()

## Data Visualization

In [ ]:
# Create a histogram for each column
df.hist(figsize=(10, 10))
plt.show()

In [ ]:
import matplotlib.pyplot as plt

df_copy = df.drop(columns='CustomerID')

# Creating multiple boxplots, one for each column
plt.figure(figsize=(10, 10))
for i, col in enumerate(df_copy.columns):
    plt.subplot(3, 3, i + 1)
    sns.boxplot(x=df_copy[col])
    plt.title(col)
plt.tight_layout()
plt.show()

In [ ]:
# Creating a correlation matrix and visualize it with a heatmap
corr = df_copy.corr()
plt.figure(figsize=(10, 6))
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

We can see a quite high correlation between days_between_trans and repeat_share here (-0.67) and a higher correlation between days_between_trans and dog_share (0.41).

In [ ]:
# Creating pairplots for all numerical data
sns.pairplot(df)
plt.show()

## Feature Relationships

In [ ]:
# Having a closer look at the most outstanding relationships
plt.figure(figsize=(10, 6))
sns.scatterplot(x='num_transactions', y='total_order_value', data=df)
plt.title('num_transactions vs. total_order_value')
plt.show()

plt.figure(figsize=(10, 6))
sns.scatterplot(x='days_between_trans', y='repeat_share', data=df)
plt.title('days_between_trans vs. repeat_share')
plt.show()

plt.figure(figsize=(10, 6))
sns.scatterplot(x='days_between_trans', y='dog_share', data=df)
plt.title('dog_share vs. repeat_share')
plt.show()

plt.figure(figsize=(10, 6))
sns.scatterplot(x='dog_share', y='repeat_share', data=df)
plt.title('dog_share vs. repeat_share')
plt.show()

# Feature Engineering

In [ ]:
## Scaling Features
# Create a copy of the dataframe
df_scaled = df.copy()

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the data
df_scaled[['num_transactions', 'total_order_value', 'days_between_trans', 'repeat_share', 'dog_share']] = scaler.fit_transform(df[['num_transactions', 'total_order_value', 'days_between_trans', 'repeat_share', 'dog_share']])
df_scaled

In [ ]:
# Check the summary statistics of the scaled data
df_scaled.describe()

## Dimensionality Reduction (optional - check model results)

In [ ]:
# The correlation between days_between_trans and repeat_share is quite high, so it might be good to use PCA to reduce the dimensionality of the data
# The problem is the interpretation of the data. Maybe simply take the two and make one out of them with PCA and keep the other 4

# Use PCA to reduce the dimensionality of the data
from sklearn.decomposition import PCA

# Initialize the PCA
pca = PCA(n_components=4)

# Fit and transform the data
df_pca = pca.fit_transform(df_scaled[['num_transactions', 'total_order_value', 'days_between_trans', 'repeat_share', 'dog_share']])
df_pca = pd.DataFrame(data=df_pca, columns=['PCA1', 'PCA2', 'PCA3', 'PCA4'])
df_pca

In [ ]:
# Check the explained variance ratio
print('Explained Variance Ratio:', pca.explained_variance_ratio_)

# Concatenate the PCA components with the original data
df_final = pd.concat([df_scaled['CustomerID'], df_pca], axis=1)
df_final

# Clustering and Segmentation

In [ ]:
#TODO: Try out more Cluster algorithms and see which one fits the best
#TODO: Try out more Dimensionality Reduction algorithms and see which one fits the best
#TODO: Try to generate more features and see if the model improves
#TODO: Try different number of clusters to find a better optimum (Elbow Method or Silhouette Score)

We can use a 3 cluster segmentation in which we describe a high quality, medium quality and low quality customer.

The high quality customer is a customer that has a high number of transactions, a high total order value, a low days between transactions, a high repeat share and a high dog share.

The low and medium quality customer accordingly. We create a lead score for each customer based on the above features and then segment the customers into 3 clusters.

We can then adjust our marketing strategy to target the high quality customers more effectively.

## Choosing the Clustering Algorithm

In [ ]:
# First try the k-means clustering algorithm
from sklearn.cluster import KMeans

# Create a copy of the final dataframe and drop the CustomerID column
df_scaled_cluster = df_scaled.copy().drop('CustomerID', axis=1)

# Initialize the KMeans algorithm
kmeans = KMeans(n_clusters=3, random_state=42)

# Fit the algorithm to the data
df_scaled_cluster['Cluster'] = kmeans.fit_predict(df_scaled_cluster)
df_scaled_cluster

# Evaluation of Segments

## Analyzing Cluster Characteristics

In [ ]:
# Calculate the mean of the clusters
cluster_stats = df_scaled_cluster.groupby('Cluster').mean()
cluster_stats

In [ ]:
# Calculate the number of customers in each cluster
cluster_size = df_scaled_cluster['Cluster'].value_counts().reset_index()
cluster_size.columns = ['Cluster', 'Count']

# Calculate the distribution of each cluster
cluster_dist = cluster_size['Count'] / cluster_size['Count'].sum()
cluster_size['Distribution'] = cluster_dist
cluster_size

## Model Validation

### Calculating Commonly Used Scores

In [ ]:
# Calculate the silhouette score
from sklearn.metrics import silhouette_score

silhouette_score(df_scaled_cluster.drop('Cluster', axis=1), df_scaled_cluster['Cluster'])

A score of more than 0.5 indicates a high-quality cluster. In our case it of course depends on the application of our clusters. If we are looking for a small number of high-quality customers, the results indicate that we could have already found them. Lets check the results further.

In [ ]:
# Calculate the Davies-Bouldin Index
from sklearn.metrics import davies_bouldin_score

davies_bouldin_score(df_scaled_cluster.drop('Cluster', axis=1), df_scaled_cluster['Cluster'])

### Visualizing the Clusters

In [ ]:
# Visualize results of the clustering by using the Cluster column and the num_transactions and total_order_value columns
plt.figure(figsize=(10, 6))
sns.scatterplot(x='repeat_share', y='dog_share', hue='Cluster', data=df_scaled_cluster, palette='viridis')
plt.title('KMeans Clustering Results')
plt.show()

In [ ]:
# Visualize results of the clustering by using the Cluster column and the num_transactions and total_order_value columns
plt.figure(figsize=(10, 6))
sns.scatterplot(x='num_transactions', y='total_order_value', hue='Cluster', data=df_scaled_cluster, palette='viridis')
plt.title('KMeans Clustering Results')
plt.show()

In [ ]:
# Visualize results of the clustering by using the Cluster column and the num_transactions and total_order_value columns
plt.figure(figsize=(10, 6))
sns.scatterplot(x='total_order_value', y='days_between_trans', hue='Cluster', data=df_scaled_cluster, palette='viridis')
plt.title('KMeans Clustering Results')
plt.show()

We can clearly see already that the clustering gives us a good segmentation of the customers. It is especially helpful to find the high quality customers that we want to explicitly target.

In [ ]:
# Add the Cluster column to the final dataframe
df_final['Cluster'] = df_scaled_cluster['Cluster']

# Visualize the cluster plots using PCA
plt.figure(figsize=(10, 6))
sns.scatterplot(x='PCA1', y='PCA2', hue='Cluster', data=df_final, palette='viridis')
plt.title('KMeans Clustering Results with PCA')
plt.show()

In [ ]:
# # Visualize the cluster plots using t-SNE
# from sklearn.manifold import TSNE

# # Initialize the t-SNE algorithm
# tsne = TSNE(n_components=2, random_state=42)

# # Fit and transform the data
# df_tsne = tsne.fit_transform(df_scaled_cluster.drop('Cluster', axis=1))

# # Create a dataframe with the t-SNE components
# df_tsne = pd.DataFrame(data=df_tsne, columns=['t-SNE1', 't-SNE2'])

# # Concatenate the t-SNE components with the cluster column
# df_tsne = pd.concat([df_tsne, df_scaled_cluster['Cluster']], axis=1)

# # Visualize the cluster plots using t-SNE
# plt.figure(figsize=(10, 6))
# sns.scatterplot(x='t-SNE1', y='t-SNE2', hue='Cluster', data=df_tsne, palette='viridis')
# plt.title('KMeans Clustering Results with t-SNE')
# plt.show()

In [ ]:
# Use plotly to create a 3D scatter plot of the clusters
fig = px.scatter_3d(df_scaled_cluster, x='num_transactions', y='total_order_value', z='days_between_trans', color='Cluster', opacity=0.7)
fig.update_layout(title='KMeans Clustering Results in 3D')
fig.show()

In [ ]:
# Use plotly to create a 3D scatter plot of the clusters
fig = px.scatter_3d(df_scaled_cluster, x='num_transactions', y='repeat_share', z='dog_share', color='Cluster', opacity=0.7)
fig.update_layout(title='KMeans Clustering Results in 3D')
fig.show()

In [ ]:
# Use plotly to create a 3D scatter plot of the clusters
fig = px.scatter_3d(df_scaled_cluster, x='num_transactions', y='days_between_trans', z='repeat_share', color='Cluster', opacity=0.7)
fig.update_layout(title='KMeans Clustering Results in 3D')
fig.show()

In this section we closely looked at the clusters and the characteristics of the customers in each cluster. We also validated the model by calculating commonly used scores and visualizing the clusters.

# Final Output

In [ ]:
# Merge df_scaled['CustomerID'] with df_scaled_cluster['Cluster']
df_clustered = pd.concat([df_scaled['CustomerID'], df_scaled_cluster['Cluster']], axis=1)
df_clustered

In [ ]:
# Save the clustered data to a CSV file
df_clustered.to_csv('Clustered_Data.csv', index=False)

# Conclusion and Recommendations

In [ ]:
#TODO: Make create a Report that looks professional and is easy to understand

## Business Insights

## Next Steps